# Calling API to retrieve JSONs for exchange rate on historical dates

In [19]:
import os
import pandas as pd
from zipfile import ZipFile
import datetime
import requests
import json

In [20]:
# setting up csv to be able to retrieve information

zf = ZipFile('archive.zip', 'r')
zf.extract('wfp_food_prices_database.csv')
zf.close()

df = pd.read_csv('wfp_food_prices_database.csv')
os.remove('wfp_food_prices_database.csv')
df.head(5)

c:\Users\Lilly\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,1,2014,50.0,NaN
1,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,2,2014,50.0,NaN
2,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,3,2014,50.0,NaN
3,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,4,2014,50.0,NaN
4,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,5,2014,50.0,NaN


In [28]:
# Create Datetime object for each row
# Isolate just the month and date of each row
tempDayMonthDF = df.loc[:, 'mp_month':'mp_year']

# Define a temp method for creating a datetime object from a row of our dataframe
def _create_datetime(row):
    return datetime.date(row[1], row[0], 1)

# Add new datetime row to the dataframe
tempDayMonthDF = tempDayMonthDF.apply(_create_datetime, axis=1)
df.insert(0, 'Date', tempDayMonthDF)
df.head()

,Date,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,2014-01-01,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,1,2014,50.0,NaN
1,2014-02-01,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,2,2014,50.0,NaN
2,2014-03-01,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,3,2014,50.0,NaN
3,2014-04-01,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,4,2014,50.0,NaN
4,2014-05-01,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,5,2014,50.0,NaN


In [42]:
# retrieve unique currencies from database
symbols = list(df['cur_name'].unique())
symbols.sort()
symbolsString = ', '.join(symbols)
symbols


['AFN',
 'AMD',
 'AOA',
 'ARS',
 'AZN',
 'BDT',
 'BIF',
 'BOB',
 'BTN',
 'BYR',
 'CDF',
 'CNY',
 'COP',
 'CVE',
 'DJF',
 'DOP',
 'DZD',
 'EGP',
 'ERN',
 'ETB',
 'GEL',
 'GHS',
 'GMD',
 'GNF',
 'GTQ',
 'HNL',
 'HTG',
 'IDR',
 'INR',
 'IQD',
 'IRR',
 'JOD',
 'JPY',
 'KES',
 'KGS',
 'KHR',
 'KZT',
 'LAK',
 'LBP',
 'LKR',
 'LRD',
 'LSL',
 'LYD',
 'MDL',
 'MGA',
 'MMK',
 'MNT',
 'MRO',
 'MWK',
 'MXN',
 'MZN',
 'NAD',
 'NGN',
 'NIO',
 'NIS',
 'NPR',
 'PAB',
 'PEN',
 'PHP',
 'PKR',
 'PYG',
 'RUB',
 'RWF',
 'SDG',
 'SLL',
 'SOS',
 'SSP',
 'SYP',
 'SZL',
 'THB',
 'TJS',
 'TRY',
 'TZS',
 'UAH',
 'UGX',
 'USD',
 'VEF',
 'VND',
 'XAF',
 'XOF',
 'YER',
 'ZAR',
 'ZMW']

In [31]:
# retrieve all unique dates in or after 1999 from database
uniqueDates = list(df['Date'].unique())
uniqueDates
kkkpast_1999Dates = list(filter(lambda x: x.year >= 1999, uniqueDates))
past_1999Dates
min(past_1999Dates)

datetime.date(1999, 1, 1)

In [22]:
#testing format of converting date to string
tempDate = min(past_1999Dates)
tempDate.strftime('%Y-%m-%d')

/


In [36]:
# setting up API calling parameters
url = 'http://api.exchangeratesapi.io/v1/'
keys = ['','']
base = 'USD'
keyNum = 0
counter = 0
# **note**: exchangerates api only allows for 250 calls per free account, therefore we need to use multiple keys to make 260 calls

responsesDict = dict() 

for date in uniqueDates:
    dateString = date.strftime('%Y-%m-%d')
    requestString ="{url}/{date}?access_key={api_key}&base=USD&symbols={symbols}".format(url=url,date=date,api_key=keys[keyNum],symbols=symbolsString)
    responseJson = requests.get(requestString).json()
    counter+=1
    if(counter == len(uniqueDates)/2):
        keyNum+=1
    responsesDict[date.strftime('%Y-%m-%d')]=responseJson

with open('exchange_rates.json', 'w') as f:
    json.dump(responsesDict, f)

# Changing Retrieved Jsons into Dictionary/CSV

In [12]:
import requests

x = requests.get('https://www.w3schools.com/python/demopage.js').json()
x


{'firstname': 'John', 'lastname': 'Doe'}

In [17]:
"""
Exchange_Rates is a dictionary of all historical exchange rates to USD. Each key is a date and each value is a JSON following the format below:
{
    "success": true,
    "timestamp": 1519296206,
    "base": "EUR",
    "date": "2021-03-17",
    "rates": {
        "AUD": 1.566015,
        "CAD": 1.560132,
        "CHF": 1.154727,
        "CNY": 7.827874,
        "GBP": 0.882047,
        "JPY": 132.360679,
        "USD": 1.23396,
    [...]
    }
}
(excerpt from https://exchangeratesapi.io/documentation/)

"""
Exchange_Rates = {"2023-03-17" : {
    "success": True,
    "timestamp": 1519296206,
    "base": "EUR",
    "date": "2023-03-17",
    "rates": {
        "AUD": 1.566015,
        "CAD": 1.560132,
        "CHF": 1.154727,
        "CNY": 7.827874,
        "GBP": 0.882047,
        "JPY": 132.360679,
        "USD": 1.23396,
    }
    } 
}

# turns dictionary into JSON object (should be example as if we called the API)
with open("sample.json",'w') as outfile:
    json.dump(Exchange_Rates,outfile)



with open("sample.json") as infile:
    sample_dict = json.load(infile)

Sample2021Json = sample_dict["2023-03-17"]
print(Sample2021Json)
Sample2021Rates = Sample2021Json["rates"]
print(Sample2021Rates)

# https://api.exchangeratesapi.io/v1/2013-12-24
#     ? access_key = API_KEY
#     & base = GBP
#     & symbols = USD,CAD,EUR

# hardcode currencies

# populate dict and then dump in the following block

{'success': True, 'timestamp': 1519296206, 'base': 'EUR', 'date': '2023-03-17', 'rates': {'AUD': 1.566015, 'CAD': 1.560132, 'CHF': 1.154727, 'CNY': 7.827874, 'GBP': 0.882047, 'JPY': 132.360679, 'USD': 1.23396}}
{'AUD': 1.566015, 'CAD': 1.560132, 'CHF': 1.154727, 'CNY': 7.827874, 'GBP': 0.882047, 'JPY': 132.360679, 'USD': 1.23396}


In [ ]:
"""
We'll save the exchange rates JSON permanently as the values never change and we don't need to call the API every time we wish to use any of the exchange rate information.
"""
with open('exchange_rates.json', 'w') as f:
    json.dump(Exchange_Rates, f)

"""
The exchange rates json can now be loaded into a larger object elsewhere in other notebooks such as foodPriceAnalysis.ipynb
"""

In [61]:
df.columns
data = df[df['cur_name'].isin(['BYR', 'MRO', 'NIS', 'SLL', 'VEF'])]
display(len(data))

38312